In [1]:
import psutil
import platform
import os.path
import numpy as np
import pickle
from multiprocessing import Pool
import refnx
from refnx.analysis import CurveFitter, Objective, Parameter, process_chain
import refnx.reflect

from refnx.reflect import (
    SLD,
    Slab,
    Structure,
    ReflectModel,
    reflectivity,
    use_reflect_backend,
    available_backends,
)
from refnx.dataset import ReflectDataset as RD

In [2]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor


uname = platform.uname()
print(f"Python version: {platform.python_version()}")
print(f"numpy version: {np.version.version}")
print(f"refnx version: {refnx.version.version}")
print()
print(f"System: {uname.system}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))

svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")

Python version: 3.12.0
numpy version: 1.26.3
refnx version: 0.1.43.dev0+74a19fd

System: Darwin
Release: 23.2.0
Version: Darwin Kernel Version 23.2.0: Wed Nov 15 21:54:55 PST 2023; root:xnu-10002.61.3~2/RELEASE_ARM64_T8122
Machine: arm64
Processor: arm
Physical cores: 8
Total cores: 8
Total: 16.00GB
Available: 6.69GB


In [3]:
q = np.linspace(0.005, 0.5, 2000)
layers = np.array(
    [
        [0, 2.07, 0, 3],
        [50, 3.47, 0.0001, 4],
        [200, -0.5, 1e-5, 5],
        [50, 1, 0, 3],
        [0, 6.36, 0, 3],
    ]
)

## test reflectometry backend speed
### Threaded calculation

In [4]:
for backend in available_backends():
    print(f"{backend=}")
    with use_reflect_backend(backend) as f:
        %timeit f(q, layers)

backend='python'


/Users/andrew/Documents/Andy/programming/refnx/refnx/reflect/reflect_model.py:254: UserWarning: Using the SLOW reflectivity calculation.
  warnings.warn("Using the SLOW reflectivity calculation.")


383 µs ± 619 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
backend='c'
102 µs ± 352 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
backend='c_parratt'
95 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
backend='py_parratt'
331 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Unthreaded calculation

In [5]:
for backend in available_backends():
    print(f"{backend=}")
    with use_reflect_backend(backend) as f:
        %timeit f(q, layers, threads=1)

backend='python'
382 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
backend='c'
215 µs ± 451 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
backend='c_parratt'
188 µs ± 310 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
backend='py_parratt'
331 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Test resolution smearing speed
### Constant dq/q

In [6]:
q = np.geomspace(0.005, 0.5, 200)

In [7]:
%timeit reflectivity(q, layers)

227 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Pointwise dq/q

In [8]:
dq = 0.05 * q
%timeit reflectivity(q, layers, dq=dq)

171 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Test sampling speed

In [9]:
pth = os.path.dirname(os.path.abspath(refnx.reflect.__file__))
e361 = RD(os.path.join(pth, "test", "e361r.txt"))

sio2 = SLD(3.47, name="SiO2")
si = SLD(2.07, name="Si")
d2o = SLD(6.36, name="D2O")
polymer = SLD(1, name="polymer")

# e361 is an older dataset, but well characterised
structure361 = si | sio2(10, 4) | polymer(200, 3) | d2o(0, 3)
model361 = ReflectModel(structure361, bkg=2e-5)

model361.scale.vary = True
model361.bkg.vary = True
model361.scale.range(0.1, 2)
model361.bkg.range(0, 5e-5)
model361.dq = 5.0

# d2o
structure361[-1].sld.real.vary = True
structure361[-1].sld.real.range(6, 6.36)

p = structure361[1].thick
structure361[1].thick.vary = True
structure361[1].thick.range(5, 20)
structure361[2].thick.vary = True
structure361[2].thick.range(100, 220)

structure361[2].sld.real.vary = True
structure361[2].sld.real.range(0.2, 1.5)

# e361.x_err = None
np.random.seed(1)

objective = Objective(model361, e361)
fitter = CurveFitter(objective, nwalkers=200)
fitter.initialise("jitter")
model361.threads = 1

In [10]:
%timeit fitter.sample(100, pool=-1, verbose=False)

2.97 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit process_chain(objective, fitter.chain);

16.3 ms ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
